In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import TensorBuilder
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
from utils.pair_features import *

## Manipulate Hamiltonian into blocks

In [3]:
frames = ase.io.read("data/hamiltonian/water-hamiltonian/water_coords_1000.xyz",":100")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [4]:
#jorbs = json.load(open('data/water-hamiltonian/water_orbs.json', "r"))
jorbs = json.loads(json.load(open('data/hamiltonian/water-hamiltonian/water_orbs.json', "r")))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [5]:
hams = np.load("data/hamiltonian/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

blocks = dense_to_blocks(hams, frames, orbs)

In [6]:
cg = ClebschGordanReal(4)

In [7]:
coupled = couple_blocks(blocks, cg)

fock_bc = coupled

## Feature computation

In [8]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [9]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [10]:
rho0ij = rho0ij_builder(rascal_hypers, frames)

In [11]:
total_species = sorted(set(rhoi.keys['species_center']))
# total_species = list(np.sort(np.asarray(total_species)))
lmax=rascal_hypers["max_angular"]
nmax=rascal_hypers["max_radial"]

In [12]:
blocks = []
for l in range(lmax+1):
    for sp_i in total_species:
        for sp_k in total_species:
            n_selected = nmax#len(np.where(opt_eva[l] > sel_thresh)[0])    
            de_block = rhoi.block(species_center = sp_i, species_neighbor=sp_k, spherical_harmonics_l = l)
            block = TensorBlock(
                values = de_block.values,
                samples = de_block.samples,
                components = [Labels(["m"],np.asarray(range(-l,l+1), dtype=np.int32).reshape(-1,1))],
                properties = Labels(["n"], np.asarray([[n] for n in range(nmax)], dtype=np.int32))
            )
            
            blocks.append( block )

acdc_nu1 = TensorMap(
    keys = Labels(names=["L", "nu", "sigma","species_i", "species_neighbor"], 
                        values=np.asarray([[ l, 1, 1, sp_i, sp_k] for l in range(rascal_hypers["max_angular"]+1) 
                                                        for sp_i in total_species
                                                        for sp_k in total_species], dtype=np.int32)
                                     ), 
                      blocks = blocks
                     )
#move neighbor species to features  
# acdc_nu1.sparse_to_features('species_neighbor')

In [13]:
rho1 = acdc_nu1.keys_to_properties("species_neighbor")

In [14]:
rho1ij=tensor_g_rho_nu(rho0ij, acdc_nu1, rascal_hypers, cg)

/Users/guillaume/code/equistore-examples/utils/pair_features.py:229: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  if i not in new_sparse_labels:


(0, 0, 1, 1, 1, 1) done
(0, 1, 1, 1, 1, 1) done
(0, 1, 1, -1, 1, 1) done
(0, 2, 1, 1, 1, 1) done
(0, 2, 1, -1, 1, 1) done
(1, 0, 1, 1, 1, 1) done
(-1, 0, 1, 1, 1, 1) done
(1, 1, 1, 1, 1, 1) done
(1, 1, 1, -1, 1, 1) done
(-1, 1, 1, 1, 1, 1) done
(-1, 1, 1, -1, 1, 1) done
(1, 2, 1, 1, 1, 1) done
(1, 2, 1, -1, 1, 1) done
(-1, 2, 1, 1, 1, 1) done
(-1, 2, 1, -1, 1, 1) done
(2, 0, 1, 1, 1, 8) done
(2, 1, 1, 1, 1, 8) done
(2, 1, 1, -1, 1, 8) done
(2, 2, 1, 1, 1, 8) done
(2, 2, 1, -1, 1, 8) done
(0, 0, 1, 1, 8, 8) done
(0, 1, 1, 1, 8, 8) done
(0, 1, 1, -1, 8, 8) done
(0, 2, 1, 1, 8, 8) done
(0, 2, 1, -1, 8, 8) done


In [15]:
feats = rho1ij

## Testing model on one structure

In [16]:
from utils.Hamiltonian_model_utils import *

In [17]:
FR = Fock_regression(jorbs)

In [18]:
FR.fit(feats, fock_bc)

In [19]:
pred = FR.predict(feats)

In [20]:
pred.keys

Labels([(0, 8, 2, 0, 8, 2, 0, 0), (0, 8, 2, 0, 8, 2, 1, 1),
        (0, 8, 2, 1, 8, 2, 1, 0), (0, 8, 2, 1, 8, 2, 1, 2),
        (2, 1, 1, 0, 8, 2, 0, 0), (2, 1, 1, 0, 8, 2, 1, 1),
        (0, 1, 1, 0, 1, 1, 0, 0), (1, 1, 1, 0, 1, 1, 0, 0)],
       dtype=[('block_type', '<i4'), ('a_i', '<i4'), ('n_i', '<i4'), ('l_i', '<i4'), ('a_j', '<i4'), ('n_j', '<i4'), ('l_j', '<i4'), ('L', '<i4')])

In [21]:
for i in range(len(fock_bc.keys)):
    print(pred.block(i).values - fock_bc.block(i).values)

[[[0.83320033]]

 [[0.77264659]]

 [[0.85020214]]

 [[0.75582823]]

 [[0.82976599]]

 [[0.82299416]]

 [[0.96293636]]

 [[0.84909555]]

 [[0.8932615 ]]

 [[0.86456442]]

 [[0.85991824]]

 [[0.7966822 ]]

 [[0.85544665]]

 [[0.84246757]]

 [[0.84916739]]

 [[0.87482555]]

 [[0.90816664]]

 [[0.85020193]]

 [[0.85020188]]

 [[0.80522904]]

 [[0.85020195]]

 [[0.80743255]]

 [[0.94418954]]

 [[0.81210895]]

 [[0.76567719]]

 [[0.89913634]]

 [[0.82859379]]

 [[0.92471482]]

 [[0.91039096]]

 [[0.87323682]]

 [[0.93930338]]

 [[0.91855832]]

 [[0.91778139]]

 [[0.89213717]]

 [[0.85020173]]

 [[0.82252199]]

 [[0.74662393]]

 [[0.87110145]]

 [[0.8502019 ]]

 [[0.85521101]]

 [[0.95353383]]

 [[0.85371249]]

 [[0.90263076]]

 [[0.97609227]]

 [[0.92177016]]

 [[0.76571431]]

 [[0.81841861]]

 [[0.85538952]]

 [[0.78484156]]

 [[0.88035063]]

 [[0.76614807]]

 [[0.81620147]]

 [[0.87272659]]

 [[0.90535962]]

 [[0.8178315 ]]

 [[0.92116411]]

 [[0.74005628]]

 [[0.93245878]]

 [[0.9687021 ]